Previously, models were saved via `ModelCheckpoint`.
In this notebook, I save the model into JSON architecture and weights file 
in an attempt to lower the amount of memory it consumes when loading the model
to Jetson TX2. 

Reference 1: https://stackoverflow.com/questions/59213329/tensorflow-resnet-model-loading-uses-5-gb-of-ram-while-loading-from-weigh
Reference 2: https://machinelearningmastery.com/save-load-keras-deep-learning-models/

In [2]:
import os
import glob
import json
import numpy as np
import cv2
from keras.applications.resnet50 import preprocess_input
from keras.models import load_model
np.random.seed(251)

# Below is needed to load model
from keras_efficientnets import EfficientNetB0

Using TensorFlow backend.


In [3]:
##################################### NEW4_trial4_model2 #####################################
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
dir_openpose = 'openpose_output'

# Parameters needed for model scoring
model_name = 'Efficientnet_model_weights_NEW4_trial4_model2'
model = load_model(os.path.join(project_path, model_name+'.h5'))

In [4]:
# serialize model to JSON
model_json = model.to_json()
with open(os.path.join(project_path, model_name+'.json'), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(project_path, model_name+'_only.h5'))
print("Saved model to disk")


Saved model to disk


In [4]:
from keras.preprocessing import image
import numpy as np
from keras.models import model_from_json
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
dir_openpose = 'openpose_output'
dir_test = 'NEW4_manual_optical_flow_output_trial4'

class_list = ['AGAIN', 'ALL', 'AWKWARD', 'BASEBALL', 'BEHAVIOR', 'CAN', 'CHAT', 'CHEAP', 
              'CHEAT', 'CHURCH', 'COAT', 'CONFLICT', 'COURT', 'DEPOSIT', 'DEPRESS', 
              'DOCTOR', 'DRESS', 'ENOUGH', 'NEG']
model_name = 'Efficientnet_model_weights_NEW4_trial4'
HEIGHT = 224
WIDTH = 224
# load json and create model
json_file = open(os.path.join(project_path, model_name+'.json'), 'r')
loaded_model_json = json_file.read()
json_file.close()
model_saved = model_from_json(loaded_model_json)
# load weights into new model
model_saved.load_weights(os.path.join(project_path, model_name+'_only.h5'))
print("Loaded model from disk")

def conv_index_to_vocab(ind):
    temp_dict = dict(enumerate(class_list))
    return temp_dict[ind]
def conv_vocab_to_index(vocab):
    temp_dict = dict(zip(class_list,range(len(class_list))))
    return temp_dict[vocab]

print(conv_index_to_vocab(0))
print(conv_vocab_to_index('NEG'))

Loaded model from disk
AGAIN
18


In [5]:
# Check if accuracy suffers from loading model from JSON
# The calculated accuracy is the same

correct_count = 0
count = 0
test_files_lst = [f for f in os.listdir(os.path.join(project_path, dir_openpose, dir_test)) 
                  if 'test' in f and 'sim0' in f]
# test_files_lst = [f for f in os.listdir(os.path.join(project_path, dir_openpose, dir_test)) 
#                   if 'test' in f]
for file in test_files_lst:
    img = image.load_img(os.path.join(project_path,dir_openpose, dir_test,file), target_size=(HEIGHT, WIDTH))
    x = image.img_to_array(img)
    # print(x.shape)
    x = np.expand_dims(x, axis=0)
    # print(x.shape)
    x = preprocess_input(x)
    y_pred = model_saved.predict(x)
    print('-----------------')
    print('Actual: ', file.split('_')[1])
    print('Prediction: ', conv_index_to_vocab(np.argmax(y_pred)))
    # print(y_pred)
    count += 1
    if file.split('_')[1] == conv_index_to_vocab(np.argmax(y_pred)):
        correct_count += 1 

-----------------
Actual:  AGAIN
Prediction:  AGAIN
-----------------
Actual:  ALL
Prediction:  ENOUGH
-----------------
Actual:  AWKWARD
Prediction:  AWKWARD
-----------------
Actual:  BASEBALL
Prediction:  ENOUGH
-----------------
Actual:  BEHAVIOR
Prediction:  BEHAVIOR
-----------------
Actual:  CAN
Prediction:  CAN
-----------------
Actual:  CHAT
Prediction:  COURT
-----------------
Actual:  CHAT
Prediction:  CHAT
-----------------
Actual:  CHEAP
Prediction:  CHEAP
-----------------
Actual:  CHEAT
Prediction:  CHEAT
-----------------
Actual:  CHEAT
Prediction:  BEHAVIOR
-----------------
Actual:  CHURCH
Prediction:  CHURCH
-----------------
Actual:  COAT
Prediction:  COAT
-----------------
Actual:  COAT
Prediction:  ALL
-----------------
Actual:  CONFLICT
Prediction:  CONFLICT
-----------------
Actual:  COURT
Prediction:  COURT
-----------------
Actual:  DEPOSIT
Prediction:  DRESS
-----------------
Actual:  DEPRESS
Prediction:  DEPRESS
-----------------
Actual:  DOCTOR
Prediction: 

In [6]:
print('Accuracy = ', correct_count/len(test_files_lst))

Accuracy =  0.6956521739130435
